<a href="https://colab.research.google.com/github/Rekyb415/Stock-Price-Prediction/blob/main/stock_price_prediction_LR_%26_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dùng Logistics Regression để dự đoán giá cổ phiếu lên hay xuống cho ngày kế tiếp

- Mô hình này là một thuật toán máy học cho các vấn đề về phân loại
- Dự đoán đầu ra là nhị phân


# KHAI BÁO THƯ VIỆN

In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 7.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import statsmodels.api as sm # phân tích thống kê
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import yfinance as yf
from sklearn.metrics import precision_score

import datetime

# Load dữ liệu trực tiếp

In [ ]:
yf.pdr_override()

#Lấy dữ liệu từ datasource

In [ ]:
from pandas_datareader import data

In [ ]:
start_date = datetime.datetime(2012,11,2)
end_date = datetime.datetime(2022,11,2)

Lấy dữ liệu chứng khoán trong 3 năm của Microsoft từ Yahoo Finance. Có thể lấy dữ liệu từ google, fred, quandl, worldbank

#LR

#Microsoft

## Chuẩn bị dữ liệu

In [ ]:
MSFT_data = pdr.get_data_yahoo('MSFT', start=start_date, end=end_date)
MSFT_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-11-02,29.590000,29.770000,29.330000,29.500000,24.126631,57131600
2012-11-05,29.620001,29.740000,29.330000,29.629999,24.232946,38070800
2012-11-06,29.820000,30.200001,29.610001,29.860001,24.421057,43401500
2012-11-07,29.530001,29.830000,29.049999,29.080000,23.783127,57871800
2012-11-08,29.120001,29.370001,28.799999,28.809999,23.562307,49841800
...,...,...,...,...,...,...
2022-10-26,231.169998,238.300003,230.059998,231.320007,230.669937,82543200
2022-10-27,231.039993,233.690002,225.779999,226.750000,226.112778,40424600
2022-10-28,226.240005,236.600006,226.050003,235.869995,235.207138,40647700


Nếu giá tăng, phần trăm thay đổi được tính bằng công thức: [(Giá mới - Giá cũ) / Giá cũ] rồi nhân số đó với 100. Nếu giá giảm, phần trăm thay đổi được tính bằng công thức [(Giá cũ - Giá mới) / Giá cũ] và nhân số đó với 100.

Công thức này được sử dụng để theo dõi giá của chứng khoán riêng lẻ và các chỉ số thị trường lớn

In [ ]:
MSFT = MSFT_data["Adj Close"].pct_change() * 100 #Lấy cột Adj Close và
                                                # Hàm pct: để lấy tiền lãi hằng ngày dựa trên Adj Close
                                                 # Và nhân với 100 vì lấy các giá trị theo tỷ lệ phần trăm


In [ ]:
MSFT

Date
2012-11-02         NaN
2012-11-05    0.440657
2012-11-06    0.776259
2012-11-07   -2.612213
2012-11-08   -0.928471
                ...   
2022-10-26   -7.715629
2022-10-27   -1.975619
2022-10-28    4.022046
2022-10-31   -1.585614
2022-11-01   -1.705939
Name: Adj Close, Length: 2517, dtype: float64

In [ ]:
MSFT = MSFT.rename("Today") # Cột Today chứa thay đổi giá tương đối trong ngày

In [ ]:
MSFT

Date
2012-11-02         NaN
2012-11-05    0.440657
2012-11-06    0.776259
2012-11-07   -2.612213
2012-11-08   -0.928471
                ...   
2022-10-26   -7.715629
2022-10-27   -1.975619
2022-10-28    4.022046
2022-10-31   -1.585614
2022-11-01   -1.705939
Name: Today, Length: 2517, dtype: float64

In [ ]:
MSFT = MSFT.reset_index()

In [ ]:
MSFT

,Date,Today
0,2012-11-02,NaN
1,2012-11-05,0.440657
2,2012-11-06,0.776259
3,2012-11-07,-2.612213
4,2012-11-08,-0.928471
...,...,...
2512,2022-10-26,-7.715629
2513,2022-10-27,-1.975619
2514,2022-10-28,4.022046
2515,2022-10-31,-1.585614


In [ ]:
# Lags chỉ là phần trăm thay đổi của ngày hôm trước (chứa dữ liệu của những ngày trc đó)=> Lag 1 là phần trăm thay đổi của ngày trước đó
for i in range(1,8):
  MSFT['Lag '+str(i)] = MSFT['Today'].shift(i)

In [ ]:
MSFT

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7
0,2012-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-11-05,0.440657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-11-06,0.776259,0.440657,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-11-07,-2.612213,0.776259,0.440657,NaN,NaN,NaN,NaN,NaN
4,2012-11-08,-0.928471,-2.612213,0.776259,0.440657,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,3.924572
2513,2022-10-27,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368
2514,2022-10-28,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962
2515,2022-10-31,-1.585614,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546


In [ ]:
MSFT['Volume'] = MSFT_data.Volume.shift(1).values/1000_000_000 #tương đồng với các Lag

In [ ]:
MSFT = MSFT.dropna() #để dữ liệu dễ sử dụng

In [ ]:
MSFT

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume
8,2012-11-14,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.776259,0.440657,0.131689
9,2012-11-15,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.776259,0.076086
10,2012-11-16,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.050956
11,2012-11-19,0.791857,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,0.064083
12,2012-11-20,-0.074820,0.791857,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,0.057179
...,...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,3.924572,0.034776
2513,2022-10-27,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,0.082543
2514,2022-10-28,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.040425
2515,2022-10-31,-1.585614,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,0.040648


In [ ]:
MSFT['Target'] = [1 if i > 0 else 0 for i in MSFT['Today']] # Cột Target(hướng) biển hiện giá trị tăng hoặc giảm

<ipython-input-18-e08af76eef53>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MSFT['Target'] = [1 if i > 0 else 0 for i in MSFT['Today']] # Cột Target(hướng) biển hiện giá trị tăng hoặc giảm


In [ ]:
MSFT # khung dữ liệu này là 1 phần của thư viện ISLR, chứa lợi nhuận hàng ngày của S&P 500 từ 2012 đến 2022   

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume,Target
8,2012-11-14,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.776259,0.440657,0.131689,0
9,2012-11-15,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.776259,0.076086,0
10,2012-11-16,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.050956,0
11,2012-11-19,0.791857,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,0.064083,1
12,2012-11-20,-0.074820,0.791857,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,0.057179,0
...,...,...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,3.924572,0.034776,0
2513,2022-10-27,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,0.082543,0
2514,2022-10-28,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.040425,1
2515,2022-10-31,-1.585614,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,0.040648,0


In [ ]:
MSFT = sm.add_constant(MSFT) # Trong mô hình thống kê api cần 1 hằng số(const) vào dataframe vì nếu không thêm hằng số, hồi quy không có hệ số chặn

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
MSFT

,const,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume,Target
8,1.0,2012-11-14,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.776259,0.440657,0.131689,0
9,1.0,2012-11-15,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.776259,0.076086,0
10,1.0,2012-11-16,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,-2.612213,0.050956,0
11,1.0,2012-11-19,0.791857,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,-0.928471,0.064083,1
12,1.0,2012-11-20,-0.074820,0.791857,-0.525132,-0.670620,-0.922881,-3.215418,-2.115861,0.069430,0.057179,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2512,1.0,2022-10-26,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,3.924572,0.034776,0
2513,1.0,2022-10-27,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.408368,0.082543,0
2514,1.0,2022-10-28,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,-0.846962,0.040425,1
2515,1.0,2022-10-31,-1.585614,4.022046,-1.975619,-7.715629,1.379166,2.118791,2.528052,-0.139546,0.040648,0


##Train mô hình

xác định các biến độc lập và các biến phụ thuộc của mô hình này



Tác giả của cuốn sách dùng Lag và Volume để dự đoán Target, vì vậy trong Logistic Regression này:
- biến độc lập là Lag và Volume
- biến phụ thuộc là Target

In [ ]:
# X là các biến độc lập
X = MSFT[['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7','Volume']]

In [ ]:
# y là biến phụ thuộc
y = MSFT.Target

chia tập train và test

In [ ]:
# Sử dụng dữ liệu trước năm 2022, kiểm tra mô hình dự đoán gì trong năm 2022 và so sanh với các giá trị thực tế ở năm 2022
X_train = MSFT[MSFT.Date.dt.year < 2022][['const', 'Lag 1', 'Lag 2','Lag 3','Lag 4', 'Lag 5', 'Lag 6', 'Lag 7','Volume']]
y_train = MSFT[MSFT.Date.dt.year < 2022]['Target']
X_test = MSFT[MSFT.Date.dt.year == 2022][['const', 'Lag 1', 'Lag 2', 'Lag 3','Lag 4', 'Lag 5', 'Lag 6', 'Lag 7','Volume']]
y_test = MSFT[MSFT.Date.dt.year == 2022]['Target']

In [ ]:
# xác định mô hình bằng dữ liệu huấn luyện
model = sm.Logit(y_train,X_train)

In [ ]:
# cho kết quả, từ đó điều chỉnh mô hình cho phù hợp với các biến
result = model.fit()

Optimization terminated successfully.
         Current function value: 0.685570
         Iterations 5


In [ ]:
# Dự đoán, so sánh với y_test
prediction = result.predict(X_test)

In [ ]:
# Ma trận nhầm lẫn, So Sánh các giá trị thực với các giá trị dự đoán
# Trong chức năng này đang thực hiện phép biến đổi của những dự đoán này(prediction)
def confusion_matrix(act,pred):# thêm đối số
  predtrans = ['Up' if i > 0.5 else "Down" for i in pred]#chuyển đổi các giá trị prediction(dự đoán)
  actuals = ['Up' if i > 0 else "Dowm" for i in act]#chuyển đổi các giá trị thực
  confusion_matrix = pd.crosstab(pd.Series(actuals), pd.Series(predtrans), #chức năng crosstab
                                 rownames=['Actual'],
                                 colnames=['Predict'])
  return confusion_matrix

##Độ chính xác của mô hình

In [ ]:
confusion_matrix(y_test,prediction) # cho phép hình dung hiệu suất của một thuật toán.

Predict,Down,Up
Actual,,
Dowm,23,91
Up,18,78


In [ ]:
78/(18+78)

0.8125

# Netflix

## Chuẩn bị dữ liệu

In [ ]:
NFLX_data = pdr.get_data_yahoo('NFLX', start=start_date, end=end_date)
NFLX_data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-11-02,11.035714,11.407143,10.852857,10.985714,10.985714,29650600
2012-11-05,10.652857,11.277143,10.642857,11.177143,11.177143,29016400
2012-11-06,11.008571,11.421429,10.815714,10.910000,10.910000,39102700
2012-11-07,10.918571,11.321429,10.918571,11.097143,11.097143,31099600
2012-11-08,11.164286,11.475714,10.827143,10.852857,10.852857,33381600
...,...,...,...,...,...,...
2022-10-26,290.040009,305.630005,288.040009,298.619995,298.619995,15714100
2022-10-27,298.329987,305.209991,294.779999,296.940002,296.940002,14612600
2022-10-28,297.700012,301.190002,292.290009,295.720001,295.720001,9960500


In [ ]:
NFLX = NFLX_data["Adj Close"].pct_change() * 100

In [ ]:
NFLX

Date
2012-11-02         NaN
2012-11-05    1.742528
2012-11-06   -2.390085
2012-11-07    1.715338
2012-11-08   -2.201346
                ...   
2022-10-26    2.611507
2022-10-27   -0.562585
2022-10-28   -0.410858
2022-10-31   -1.298524
2022-11-01   -1.757573
Name: Adj Close, Length: 2517, dtype: float64

In [ ]:
NFLX = NFLX.rename("Today")

In [ ]:
NFLX

Date
2012-11-02         NaN
2012-11-05    1.742528
2012-11-06   -2.390085
2012-11-07    1.715338
2012-11-08   -2.201346
                ...   
2022-10-26    2.611507
2022-10-27   -0.562585
2022-10-28   -0.410858
2022-10-31   -1.298524
2022-11-01   -1.757573
Name: Today, Length: 2517, dtype: float64

In [ ]:
NFLX = NFLX.reset_index()

In [ ]:
NFLX

,Date,Today
0,2012-11-02,NaN
1,2012-11-05,1.742528
2,2012-11-06,-2.390085
3,2012-11-07,1.715338
4,2012-11-08,-2.201346
...,...,...
2512,2022-10-26,2.611507
2513,2022-10-27,-0.562585
2514,2022-10-28,-0.410858
2515,2022-10-31,-1.298524


In [ ]:
for i in range(1,8):
  NFLX['Lag '+str(i)] = NFLX['Today'].shift(i)

In [ ]:
NFLX

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7
0,2012-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-11-05,1.742528,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-11-06,-2.390085,1.742528,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-11-07,1.715338,-2.390085,1.742528,NaN,NaN,NaN,NaN,NaN
4,2012-11-08,-2.201346,1.715338,-2.390085,1.742528,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,6.565220
2513,2022-10-27,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908
2514,2022-10-28,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442
2515,2022-10-31,-1.298524,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307


In [ ]:
NFLX['Volume'] = NFLX_data.Volume.shift(1).values/1000_000_000

In [ ]:
NFLX = NFLX.dropna()

In [ ]:
NFLX

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume
8,2012-11-14,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,-2.390085,1.742528,0.027171
9,2012-11-15,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,-2.390085,0.026911
10,2012-11-16,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,0.017401
11,2012-11-19,0.568600,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,0.032164
12,2012-11-20,1.278274,0.568600,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,0.023267
...,...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,6.565220,0.015101
2513,2022-10-27,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,0.015714
2514,2022-10-28,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,0.014613
2515,2022-10-31,-1.298524,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,0.009961


In [ ]:
NFLX['Target'] = [1 if i > 0 else 0 for i in NFLX['Today']]

<ipython-input-43-468ec9bfaca0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NFLX['Target'] = [1 if i > 0 else 0 for i in NFLX['Today']]


In [ ]:
NFLX

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume,Target
8,2012-11-14,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,-2.390085,1.742528,0.027171,1
9,2012-11-15,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,-2.390085,0.026911,1
10,2012-11-16,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,0.017401,0
11,2012-11-19,0.568600,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,0.032164,1
12,2012-11-20,1.278274,0.568600,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,0.023267,1
...,...,...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,6.565220,0.015101,1
2513,2022-10-27,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,0.015714,0
2514,2022-10-28,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,0.014613,0
2515,2022-10-31,-1.298524,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,0.009961,0


In [ ]:
NFLX = sm.add_constant(NFLX)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
NFLX

,const,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume,Target
8,1.0,2012-11-14,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,-2.390085,1.742528,0.027171,1
9,1.0,2012-11-15,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,-2.390085,0.026911,1
10,1.0,2012-11-16,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,1.715338,0.017401,0
11,1.0,2012-11-19,0.568600,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,-2.201346,0.032164,1
12,1.0,2012-11-20,1.278274,0.568600,-0.711831,2.130858,0.213546,1.816088,0.372281,2.540473,0.023267,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2512,1.0,2022-10-26,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,6.565220,0.015101,1
2513,1.0,2022-10-27,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,-1.729908,0.015714,0
2514,1.0,2022-10-28,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,13.086442,0.014613,0
2515,1.0,2022-10-31,-1.298524,-0.410858,-0.562585,2.611507,3.034157,-2.458816,7.984041,-1.549307,0.009961,0


##Train mô hình

In [ ]:
X1 = NFLX[['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7', 'Volume']]

In [ ]:
y1 = NFLX.Target

In [ ]:
def confusion_matrix(act,pred):
  predtrans = ['Up' if i > 0.5 else "Down" for i in pred]
  actuals = ['Up' if i > 0 else "Dowm" for i in act]
  confusion_matrix = pd.crosstab(pd.Series(actuals),
                                 pd.Series(predtrans),
                                 rownames=['Actual'],
                                 colnames=['Predict'])
  return confusion_matrix

chia tập dữ liệu train và test

In [ ]:
X1_train = NFLX[NFLX.Date.dt.year < 2022][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7', 'Volume']] #nếu lấy 4 lag thì độ chính xác cao nhất
y1_train = NFLX[NFLX.Date.dt.year < 2022]['Target']
X1_test = NFLX[NFLX.Date.dt.year == 2022][['const', 'Lag 1', 'Lag 2', 'Lag 7', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7', 'Volume']]
y1_test = NFLX[NFLX.Date.dt.year == 2022]['Target']

In [ ]:
model1 = sm.Logit(y1_train,X1_train)

In [ ]:
result1 = model1.fit()

Optimization terminated successfully.
         Current function value: 0.689321
         Iterations 4


In [ ]:
prediction1 = result1.predict(X1_test)

##Độ chính xác của mô hình

In [ ]:
confusion_matrix(y1_test,prediction1)

Predict,Down,Up
Actual,,
Dowm,32,85
Up,26,67


In [ ]:
67/(67+26)

0.7204301075268817

#Apple

##Chuẩn bị dữ liệu

In [ ]:
AAPL_Data = pdr.get_data_yahoo('AAPL', start=start_date, end=end_date)
AAPL_Data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-11-02,21.281786,21.319643,20.526787,20.600000,17.635199,599373600
2012-11-05,20.840000,20.991785,20.628571,20.879286,17.874290,529135600
2012-11-06,21.079643,21.097857,20.717501,20.816071,17.820171,374917200
2012-11-07,20.494286,20.519285,19.848213,19.928572,17.138327,793648800
2012-11-08,20.022499,20.079643,19.117500,19.205357,16.516373,1056146000
...,...,...,...,...,...,...
2022-10-26,150.960007,151.990005,148.039993,149.350006,149.102661,88194300
2022-10-27,148.070007,149.050003,144.130005,144.800003,144.560196,109180200
2022-10-28,148.199997,157.500000,147.820007,155.740005,155.482086,164762400


In [ ]:
AAPL = AAPL_Data["Adj Close"].pct_change() * 100

In [ ]:
AAPL

Date
2012-11-02         NaN
2012-11-05    1.355765
2012-11-06   -0.302776
2012-11-07   -3.826252
2012-11-08   -3.629024
                ...   
2022-10-26   -1.962714
2022-10-27   -3.046535
2022-10-28    7.555254
2022-10-31   -1.541040
2022-11-01   -1.754273
Name: Adj Close, Length: 2517, dtype: float64

In [ ]:
AAPL = AAPL.rename("Today")

In [ ]:
AAPL

Date
2012-11-02         NaN
2012-11-05    1.355765
2012-11-06   -0.302776
2012-11-07   -3.826252
2012-11-08   -3.629024
                ...   
2022-10-26   -1.962714
2022-10-27   -3.046535
2022-10-28    7.555254
2022-10-31   -1.541040
2022-11-01   -1.754273
Name: Today, Length: 2517, dtype: float64

In [ ]:
AAPL = AAPL.reset_index()

In [ ]:
AAPL

,Date,Today
0,2012-11-02,NaN
1,2012-11-05,1.355765
2,2012-11-06,-0.302776
3,2012-11-07,-3.826252
4,2012-11-08,-3.629024
...,...,...
2512,2022-10-26,-1.962714
2513,2022-10-27,-3.046535
2514,2022-10-28,7.555254
2515,2022-10-31,-1.541040


In [ ]:
for i in range(1,8):
  AAPL['Lag '+str(i)] = AAPL['Today'].shift(i)

In [ ]:
AAPL

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7
0,2012-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-11-05,1.355765,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-11-06,-0.302776,1.355765,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-11-07,-3.826252,-0.302776,1.355765,NaN,NaN,NaN,NaN,NaN
4,2012-11-08,-3.629024,-3.826252,-0.302776,1.355765,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,2.912274
2513,2022-10-27,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936
2514,2022-10-28,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522
2515,2022-10-31,-1.541040,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707


In [ ]:
AAPL['Volume'] = AAPL_Data.Volume.shift(1).values/1000_000_000

In [ ]:
AAPL = AAPL.dropna()

In [ ]:
AAPL

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume
8,2012-11-14,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,-0.302776,1.355765,0.532949
9,2012-11-15,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,-0.302776,0.477170
10,2012-11-16,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,0.789911
11,2012-11-19,7.210853,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,1.266894
12,2012-11-20,-0.852062,7.210853,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,0.823318
...,...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,2.912274,0.074732
2513,2022-10-27,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,0.088194
2514,2022-10-28,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.109180
2515,2022-10-31,-1.541040,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.164762


In [ ]:
AAPL['Target'] = [1 if i > 0 else 0 for i in AAPL['Today']]

<ipython-input-68-3d0be12e19ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AAPL['Target'] = [1 if i > 0 else 0 for i in AAPL['Today']]


In [ ]:
AAPL

,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume,Target
8,2012-11-14,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,-0.302776,1.355765,0.532949,0
9,2012-11-15,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,-0.302776,0.477170,0
10,2012-11-16,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,0.789911,1
11,2012-11-19,7.210853,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,1.266894,1
12,2012-11-20,-0.852062,7.210853,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,0.823318,0
...,...,...,...,...,...,...,...,...,...,...,...
2512,2022-10-26,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,2.912274,0.074732,0
2513,2022-10-27,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,0.088194,0
2514,2022-10-28,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.109180,1
2515,2022-10-31,-1.541040,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.164762,0


In [ ]:
AAPL = sm.add_constant(AAPL)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
AAPL

,const,Date,Today,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Lag 6,Lag 7,Volume,Target
8,1.0,2012-11-14,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,-0.302776,1.355765,0.532949,0
9,1.0,2012-11-15,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,-0.302776,0.477170,0
10,1.0,2012-11-16,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,-3.826252,0.789911,1
11,1.0,2012-11-19,7.210853,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,-3.629024,1.266894,1
12,1.0,2012-11-20,-0.852062,7.210853,0.391918,-2.097284,-1.108911,0.012927,-0.773289,1.731334,0.823318,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2512,1.0,2022-10-26,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,2.912274,0.074732,0
2513,1.0,2022-10-27,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.940936,0.088194,0
2514,1.0,2022-10-28,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.076522,0.109180,1
2515,1.0,2022-10-31,-1.541040,7.555254,-3.046535,-1.962714,1.933764,1.480265,2.705912,-0.326707,0.164762,0


##Train mô hình

In [ ]:
X2 = AAPL[['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7', 'Volume']]

In [ ]:
y2 = AAPL.Target

In [ ]:
def confusion_matrix(act,pred):
  predtrans = ['Up' if i > 0.5 else "Down" for i in pred]
  actuals = ['Up' if i > 0 else "Dowm" for i in act]
  confusion_matrix = pd.crosstab(pd.Series(actuals),
                                 pd.Series(predtrans),
                                 rownames=['Actual'],
                                 colnames=['Predict'])
  return  

chia tap du lieu

In [ ]:
X2_train = AAPL[AAPL.Date.dt.year < 2022][['const', 'Lag 1', 'Lag 2', 'Lag 3', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7', 'Volume']] #nếu lấy 4 lag thì độ chính xác cao nhất
y2_train = AAPL[AAPL.Date.dt.year < 2022]['Target']
X2_test = AAPL[AAPL.Date.dt.year == 2022][['const', 'Lag 1', 'Lag 2', 'Lag 7', 'Lag 4', 'Lag 5', 'Lag 6', 'Lag 7', 'Volume']]
y2_test = AAPL[AAPL.Date.dt.year == 2022]['Target']

In [ ]:
model2 = sm.Logit(y2_train, X2_train)

In [ ]:
result2 = model2.fit()

Optimization terminated successfully.
         Current function value: 0.687797
         Iterations 4


In [ ]:
prediction2 = result2.predict(X2_test)

##Độ chính xác của mô hình

In [ ]:
confusion_matrix(y2_test,prediction2)

In [ ]:
71/(71+30)

0.7029702970297029

#KNN

#Microsoft

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knn_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

array([[48, 66],
       [45, 51]])

In [ ]:
m = precision_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

0.4358974358974359

#Netflix

In [ ]:
knn_model.fit(X1_train, y1_train)

KNeighborsClassifier()

In [ ]:
y1_pred = knn_model.predict(X1_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y1_test, y1_pred)
ac = accuracy_score(y1_test,y1_pred)

In [ ]:
confusion_matrix(y_test,y_pred)

array([[48, 66],
       [45, 51]])

In [ ]:
n = precision_score(y1_test, y1_pred)

In [ ]:
precision_score(y1_test, y1_pred)

0.423728813559322

#Apple

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn_model.fit(X2_train, y2_train)

KNeighborsClassifier()

In [ ]:
y2_pred = knn_model.predict(X2_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y2_test, y2_pred)
ac = accuracy_score(y2_test,y2_pred)

In [ ]:
confusion_matrix(y2_test,y2_pred)

array([[49, 60],
       [46, 55]])

In [ ]:
a = precision_score(y2_test, y2_pred)

In [ ]:
precision_score(y2_test, y2_pred)

0.4782608695652174

#Tinh chỉnh mô hình KNN

#Microsoft

## GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {"n_neighbors": range(1, 50)}
gridsearch = GridSearchCV(knn_model, parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50)})

In [ ]:
gridsearch.best_params_

{'n_neighbors': 21}

In [ ]:
knn_model1 = KNeighborsClassifier(n_neighbors = 21, metric = 'minkowski', p = 2)
knn_model1.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=21)

In [ ]:
y_pred = knn_model1.predict(X_test)

In [ ]:
m_tc = precision_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

0.45864661654135336

## Thêm trọng số

In [ ]:
parameters = {"n_neighbors": range(1, 50), "weights": ["uniform", "distance"],}
gridsearch = GridSearchCV(knn_model, parameters)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 50),
                         'weights': ['uniform', 'distance']})

In [ ]:
gridsearch.best_params_

{'n_neighbors': 21, 'weights': 'distance'}

In [ ]:
knn_model2 = knn_model1 = KNeighborsClassifier(n_neighbors = 21, metric = 'minkowski', p = 2, weights = 'distance')

In [ ]:
knn_model2.fit(X_train, y_train)
y_pred = knn_model2.predict(X_test)

In [ ]:
m_tc1 = precision_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

0.46923076923076923

0.43->0.45->0.47

#Netflix

In [ ]:
knn_model.fit(X1_train, y1_train)

KNeighborsClassifier()

In [ ]:
y1_pred = knn_model.predict(X1_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


In [ ]:
precision_score(y1_test, y1_pred)

0.423728813559322

In [ ]:
knn_model1.fit(X1_train, y1_train)

KNeighborsClassifier(n_neighbors=21, weights='distance')

In [ ]:
y1_pred = knn_model1.predict(X1_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


In [ ]:
precision_score(y1_test, y1_pred)

0.4806201550387597

In [ ]:
knn_model2.fit(X1_train, y1_train)

KNeighborsClassifier(n_neighbors=21, weights='distance')

In [ ]:
y1_pred = knn_model2.predict(X1_test)
n_tc = precision_score(y1_test, y1_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


In [ ]:
precision_score(y1_test, y1_pred)

0.4806201550387597

#Apple

In [ ]:
knn_model.fit(X2_train, y2_train)
y2_pred = knn_model.predict(X2_test)
precision_score(y2_test, y2_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


0.4782608695652174

In [ ]:
knn_model1.fit(X2_train, y2_train)
y2_pred = knn_model1.predict(X2_test)
precision_score(y2_test, y2_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


0.5040650406504065

In [ ]:
knn_model2.fit(X2_train, y2_train)

KNeighborsClassifier(n_neighbors=21, weights='distance')

In [ ]:
y2_pred = knn_model2.predict(X2_test)
a_tc = precision_score(y2_test, y2_pred)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- Lag 3

  warnings.warn(message, FutureWarning)


In [ ]:
precision_score(y2_test, y2_pred)

0.5040650406504065